# Tutorial 5 - Interactive Notebooks (Advanced)

Welcome to the _Interactive Notebooks Tutorial_.
You will learn how to create clean Notebooks that allow users to interact with the data with the help of various widgets. 
In this tutorial, we provide examples on how to initialize multiple datasets, how the user can select the system or date from SMF data and get the Notebook updated automatically without rerunning it.

To start, you may refer to [ipywidgets documentation](https://ipywidgets.readthedocs.io/en/latest/).

## Dataset(s) Initialization

For a graphical interface, you need to import the ``smfexplorer.util.jupyter`` module and create an instance of the ``ConfigWidget`` class, providing a **Context** as an argument.
After the  ConfigWidget was created, you can display it by calling the IPython ``display()`` function.
> **Note**: You can specify a dataset as an argument in the ``new_context()`` function. This will make the text field of the ConfigWidget contain the predefined dataset name. 

In [ ]:
import smfexplorer
from smfexplorer.fields import SMF70S1
from smfexplorer import names
from smfexplorer.util import jupyter

# text field with predefined dataset name
# ctx = smfexplorer.new_context('YOUR.SMF.DATA')

ctx = smfexplorer.new_context()
config_widget = jupyter.ConfigWidget(ctx)
display(config_widget)

> **Hint:** In some cases you may want to work with multiple datasets. For example, in one Notebook you want to analyse LPAR utilization from  "*YOUR.SMF.SMF70*"  and Cache-Hits from "*YOUR.SMF.HIS*".  To do so, you can assign both datasets to one context, separating them with a comma ‘,’ (e.g., ``YOUR.SMF.SMF70,YOUR.SMF.HIS``). When you create a request, *IBM SMF Explorer* fetches the data for all specified datasets and concatenates the results. 

To make the user-provided dataset available to the next cell, you need to do the following:
 * Reference widget instance with ``@`` and call ``register_output()`` function that automatically reruns the cell when the dataset instance was changed
 * Create a function that takes as an argument dsn (this is our dataset instance)
 
------
Let's give it a try!

Enter a dataset name into the textfield above and press the **Init** button.
The `ConfigWidget` will automatically check the specified dataset for availability and call the `give_me_dsn()` function when a user presses `Init`.
 
As you see, the output of the cell below is changed when the dataset name is changed. 

In [ ]:
@config_widget.register_output()
def give_me_dsn(dsn, **kwds):
    print(dsn)

You have seen how we can create a **Context** instance and let the user interactively define the datasets for that **Context**. 
However, we know that having dataset names is not enough.
We want to create a request and fetch a DataFrame. 
Therefore, we define another function that executes a request and returns the fetched DataFrame. 

The `name` argument in ``@config_widget.register_output(name="df")`` denotes that the function returns the DataFrame as the name **df**.
We will see why this is useful in the next cell.

In [ ]:
@config_widget.register_output(name="df")
def fetch_df(dsn, **kwds):
    df = ctx.samples.lpar_information().run()
    return df

To make use of our  **df** DataFrame, we can register the next function not with `config_widget` but with `fetch_df` (the function defined above).

Now, we can see what the `name` argument from `fetch_df` does.
By default, *IBM SMF Explorer* uses the function `name` as the name for the parameter that we want to pass down the chain.
The `name` argument makes  *IBM SMF Explorer* change `fetch_df` to `df` in `working_with_df`.

This kind of chaining can be repeated with `working_with_df` and any subsequent registered function.
This allows you to define a flow of operations that should be triggered when the user presses **Init**.

In [ ]:
@fetch_df.register_output()
def working_with_df(df, **kwds):
    display(df.head())

You can see that `working_with_df` was able to render the DataFrame by calling the `display()` function.
This is the case, because `register_output` was used.
If you have a function that you know will not display anything to the Notebook, you can use the `register` function instead.

> **Note**: `register_output` creates an `Output` widget that can be dynamically updated.
> Every time a function that is registered with `register_output` is called, the `Output` widget is cleared and repopulated with the input given to the `display()` function.

## Creating Widgets

Just having the initial dataframe passed down the chain is a good start.
To be fully interactive, we sometimes need to react to specifics in the fetched data and ask the user for additional input to provide a useful output.
The following example shows  how to use IPyWidgets together with *IBM SMF Explorer*'s interactive features.

When we are working with SMF data, we often notice that one dataset can contain information from multiple systems/dates/LPARS.
To minimize data and ease processing, it makes sense to provide filters or selectors that allow the user to select a subset of the data. 

Consider the following widgets:

 * FloatSlider
 * IntProgress
 * FloatText
 * ToggleButton
 * Checkbox
 * Dropdown

See [here](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html) for more information
 

We start by importing ``ipywidgets``.
The next step is to create the widget instance (in our case _ToggleButtons_).

We can now add our widget to `register_output`.
This will result in any function that registers against `select_name` to be called with the value of the widget whenever the DataFrame changes or the user changes the selection of the widget.

In [ ]:
import ipywidgets as widgets

selection_widget = widgets.ToggleButtons()


@fetch_df.register_output(name="df", name_selected=selection_widget)
def select_name(df, **kwds):

    lpar_system_name = df[names(SMF70S1.lpar_system_name)].unique().dropna()
    selection_widget.options = lpar_system_name

    display(selection_widget)

    selection_widget.value = lpar_system_name[0]

    return df

In [ ]:
@select_name.register_output()
def filter_df(df, name_selected, **kwds):

    # The best practice is to guard against calls, where df is None or to check the input for validity in general.
    # Returning early from the function will clear the output.
    if df is None:
        return
    print("Selected LPAR-system name is: " + name_selected)
    df = df[df[names(SMF70S1.lpar_system_name)] == name_selected].reset_index(drop=True)
    display(df.head())

If you select another LPAR-system name from the selection, you should see that the table above is redrawn.
Or if you change the dataset name in the very beginning and press *Init* again the entire output will be redrawn.

> **Note**: Getting interactive Notebooks right can be very challenging. We advise you to first implement a normal sequential Notebook without widgets that you can later convert into an interactive one. The logic for desired output in every situation can be very complex (e.g. Filtering for exception cases: what happens if the specified dataset does not contain the necessary data?).